**Входные данные:**

Файл 'products_data.csv', содержащий информацию о продаваемых продуктах(id-продукта,	название продукта,	категория,	производитель)

Файл 'sales_data.csv', содержащий информацию о продажах(id-заказа, id-продукта, id-продавца, количество, цена за еденицу, общая стоимость, способ оплаты, регион)

**Задача:** проанализировать данные и дать ответы на вопросы

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

products_data = pd.read_csv('products_data.csv')
sales_data = pd.read_csv('sales_data.csv')

In [ ]:
products_data.head(5)
# id-продукта,	название продукта,	категория,	производитель

,product_id,product_name,category,manufacturer
0,101,Ноутбук HP Pavilion 15,Ноутбуки,A
1,102,Смартфон Samsung Galaxy S21,Смартфоны,B
2,103,Планшет Apple iPad Air,Планшеты,C
3,104,Наушники Sony WH-1000XM4,Наушники,D
4,105,Телевизор LG OLED CX,Телевизоры,E


In [ ]:
sales_data.head(3)
# id-заказа, id-продукта, id-продавца, количество, цена за еденицу, общая стоимость,
# способ оплаты, регион

,order_id,product_id,customer_id,order_date,quantity,price_per_unit,total_price,payment_method,region
0,1,101,1001,2022-02-15,2,500,1000,Карта,Север
1,2,102,1002,2022-03-20,1,800,800,Наличные,Юг
2,3,103,1003,2022-03-25,3,300,900,Карта,Запад


In [ ]:
merge_data = pd.merge(products_data, sales_data,
                   left_on='product_id', right_on='product_id',
                      how='inner')
merge_data.head()


,product_id,product_name,category,manufacturer,order_id,customer_id,order_date,quantity,price_per_unit,total_price,payment_method,region
0,101,Ноутбук HP Pavilion 15,Ноутбуки,A,1,1001,2022-02-15,2,500,1000,Карта,Север
1,102,Смартфон Samsung Galaxy S21,Смартфоны,B,2,1002,2022-03-20,1,800,800,Наличные,Юг
2,103,Планшет Apple iPad Air,Планшеты,C,3,1003,2022-03-25,3,300,900,Карта,Запад
3,104,Наушники Sony WH-1000XM4,Наушники,D,4,1001,2022-04-10,2,400,800,Карта,Восток
4,105,Телевизор LG OLED CX,Телевизоры,E,5,1004,2022-04-15,1,700,700,Наличные,Север


# 1) В каком регионе было продано наибольшее количество продуктов категории "Смартфоны"?

In [ ]:
task1 = merge_data.pivot_table(index='region', columns='category',
                       values='quantity', aggfunc='sum', fill_value=0)

In [ ]:
task1[task1['Смартфоны'] == task1['Смартфоны'].max()]

# 2) Какова разница между максимальной и минимальной средней ценой продуктов категории "Ноутбуки" между всеми производителями? Ответ округлите до целых.

In [ ]:
task2 = merge_data[merge_data['category'] == 'Ноутбуки']
task2

In [ ]:
task2 = task2.groupby('manufacturer').agg({'price_per_unit':'mean'})
task2

In [ ]:
task2['price_per_unit'].max() - task2['price_per_unit'].min()

# 3) Какой покупатель совершил заказы на наибольшую сумму за все время?

In [ ]:
task3 = sales_data.groupby(['customer_id']).apply(lambda x: sum(x['price_per_unit'] * x['quantity']))

In [ ]:
task3[task3 == task3.max()]

,0
customer_id,
1003,3550


# 4) Какова доля продаж продуктов категории "Планшеты" от общей выручки за все время? Ответ округлите до 2 знака после запятой.

In [ ]:
task4 = merge_data.groupby('category').apply(lambda x: sum(x['quantity'] * x['price_per_unit']))
round(task4['Планшеты'] / task4.sum(), 2)

0.25

# 5) Какова абсолютная разница в средней цене продуктов категории "Смартфоны" между двумя самыми крупными регионами по продажам?

In [ ]:
task5 =  merge_data[merge_data['category'] == 'Смартфоны'].groupby('region', as_index=False).agg({'total_price': 'sum', 'price_per_unit': 'mean'})
task5 = task5.sort_values('total_price', ascending=False).reset_index()
task5

,index,region,total_price,price_per_unit
0,3,Юг,1400,550.0
1,0,Восток,1100,275.0
2,2,Север,550,550.0
3,1,Запад,500,250.0


In [ ]:
task5.loc[0, 'price_per_unit'] - task5.loc[1, 'price_per_unit']

275.0

# 6) Сколько в среднем уникальных категорий продуктов было продано по всем регионам?

In [ ]:
task6 = merge_data.groupby('region')['category'].nunique()
task6.mean()

5.0

# 7) Какой производитель представлен в наибольшем количестве различных категорий продуктов?

In [ ]:
task7 = merge_data.groupby('manufacturer')['category'].nunique()
task7[task7 == task7.max()]

# 8) В каком месяце было продано больше всего продуктов категории "Планшеты" в регионе "Север"?

In [ ]:
task8 = merge_data[(merge_data['category'] == 'Планшеты') & (merge_data['region'] == 'Север')]
task8['month'] = pd.to_datetime(task8['order_date']).dt.month
task8 = task8.groupby('month')['quantity'].sum()
task8

# 9) Какой производитель продал продукты на наибольшую сумму в регионе "Юг" за март 2023 года?

In [ ]:
task9 = merge_data
task9['month'] = pd.to_datetime(task9['order_date']).dt.month
task9['year'] = pd.to_datetime(task9['order_date']).dt.year

task9 = task9[(task9['month'] == 3) & (task9['year'] == 2023) & (task9['region'] == 'Юг')]
task9 = task9.groupby('manufacturer').agg({'total_price': 'sum'})
task9[task9 == task9.max()]

,total_price
manufacturer,
B,700


# 10) Сколько покупателей совершили заказы на продукты хотя бы трех разных категорий?

In [ ]:
task10 = merge_data.groupby('customer_id')['category'].nunique()
len(task10[task10 > 2])

1

# 11) Какова доля продаж продуктов производителя "C" от общей выручки за все время в регионе "Север"?

In [ ]:
task10 = merge_data[merge_data['region'] == 'Север']
task10 = task10.groupby('manufacturer', as_index=False)['total_price'].sum()
task10['part'] = round(task10['total_price'] / task10['total_price'].sum(), 2)
task10

,manufacturer,total_price,part
0,A,2100,0.30
1,B,700,0.10
2,C,1850,0.27
3,D,550,0.08
4,E,1700,0.25


In [ ]:
task10.part.sum()

1.0

In [ ]:
task10[task10.manufacturer == 'C']

,manufacturer,total_price,part
2,C,1850,0.27


# 12) Какое максимальное количество продуктов было продано в один заказ?

In [ ]:
task12 = sales_data.groupby('order_id')['quantity'].sum()
task12.max()

4

# 13) Какой производитель имеет самую высокую среднюю цену продукта среди всех категорий?

In [ ]:
task13 = merge_data.groupby(['manufacturer', 'category']).agg({'price_per_unit':'mean'})
task13.idxmax()

,0
price_per_unit,"(E, Наушники)"


# 14) Каково общее количество уникальных производителей, чьи продукты были куплены менее чем в 3 регионах?

In [ ]:
task14 = merge_data.groupby('manufacturer')['region'].nunique()
task14[task14 < 3]

,region
manufacturer,


# 15) Найдите покупателей, которые купили более чем 15% уникальных связок "Категория - Производитель".

In [ ]:
connections = merge_data.groupby(['category', 'manufacturer']).count().shape[0]
task15 = merge_data.groupby(['customer_id', 'category', 'manufacturer'], as_index=False).agg({'order_id':'count'})
task15 = task15.groupby('customer_id', as_index=False)['order_id'].count()
task15['part'] = task15['order_id'] / connections
task15 = task15[task15['part'] > 0.15]

In [ ]:
task15['customer_id'].unique()

array([1001, 1003])